In [1]:
import numpy as np
import math

Microscope: [Zeiss z.1](https://applications.zeiss.com/C125792900358A3F/0/4D1D8D177F06CDF4C1257A940041002D/$FILE/EN_41_011_005_LightsheetZ1_rel2-3.pdf)

In [6]:
# Sample
sample_size = [5, 5, 10] # z, y, x (mm)

# Camera
sensor_size = [1920, 1920] # pixels
ps = 6.5

# Motion stage
stage_axial_max_velocity = 2 # mm/s
stage_axial_max_acceleration = 1e3 # mm/s
stage_axial_resolution = 200e-6 # mm
stage_axial_settle_time = 0.2    # s

# Illumination
detection_wavelength = 0.53 # um

# Objective
system_mag = 1.25 # unitless (0.36× – 2.5×, continuous)
objective_name = '5x'
objectives = {'5x' : {'na' : 0.16, 'mag' : 5},
              '10x': {'na' : 0.5,  'mag' : 20},
              '20x': {'na' : 1.0,  'mag' : 20},
              '40x': {'na' : 1.0,  'mag' : 40},
              '63x': {'na' : 1.0,  'mag' : 64}}

# Get objective parameters
mag = objectives[objective_name]['mag']
na = objectives[objective_name]['na']
effective_pixel_size = ps / (mag * system_mag)

# Axial Scan Parameters
axial_scan_overlap_factor = 0.6 # Amount (0 to 1) of overlap between frames, relative to PSF size
axial_scan_axis = 0

# Lateral Scan Parameters
lateral_scan_axes = [1, 2] # y, x
lateral_scan_overlap_factor = 0.2

camera_readout_time = 0.017 # s
camera_exposure_time = 0.1 # s
illumination_min_update_speed = 10e-6 # s

# Check camera sampling
k_max_optical = na / detection_wavelength
k_max_sensor = 1 / (2 * effective_pixel_size)
assert k_max_sensor > k_max_optical, "Maximum optical spatial frequency (%.3fum^{-1}) is greater than the system bandwidth (%.3fum^{-1})" % (k_max_optical,k_max_sensor)

## Calculate Volumetric Parameters

In [7]:
# Calculate pixel size and resolution
lateral_resolution = detection_wavelength / na
axial_resolution = detection_wavelength / (na ** 2)
print('Lateral resolution is %.4fum, axial is %.4fum' % (lateral_resolution, axial_resolution))

# Calculate number of planes in axial scan
axial_scan_increment = axial_resolution * 1e-3 * (1 - axial_scan_overlap_factor)
axial_plane_count = sample_size[axial_scan_axis] / axial_scan_increment
print('Axial scan will require %d planes' % axial_plane_count)

# Calculate number of lateral positions to scan
sample_size_lateral = [sample_size[i] for i in lateral_scan_axes]
fov = np.asarray(sensor_size) * effective_pixel_size * 1e-3

n_frames_lateral = np.ceil(np.asarray(sample_size_lateral) / (fov * 1 - lateral_scan_overlap_factor))
print('Lateral scan will require %d x %d positions' % tuple(n_frames_lateral))

Lateral resolution is 3.3125um, axial is 20.7031um
Axial scan will require 603 planes
Lateral scan will require 3 x 6 positions


## Calculate Scan Time with Stop and Stare
A useful paremeter is the ratio of motion time to readout time - if the readout time is longer than the motion time, using motion deblur won't improve our results much.

The total time per frame is determined by:

$$ t_{frame} = t_{exposure} + \max(t_{readout}, t_{motion}) $$

In the case of continuous motion (strobed or coded illumination), $\max(t_{readout}, t_{motion}) = t_{readout}$, meaning that the acquisition time is limited by the readout time. Assuming $t_{motion} > t_{readout}$, the improvement ratio of continuous scanning over conventional imaging is therefore:

$$ f = \frac{t_{exposure} + t_{motion}}{t_{exposure} + t_{readout}} $$

Using this analysis, it is clear that increasing $t_{readout}$ will decrease $f$, meaning that readout times are the enemy of motion deblur. Obviously, a higher $t_{motion}$ (slower acceleration, longer settle times) will lead to a larger $f$.

From this we can conclude that our method is best applied in situations where:
- The readout time is very short (shorter the better)
- The system has low acceleration or long settle time (lead-screw based systems)
- The exposure time is < motion time (or readout time)

In [9]:
# Ensure state increment is not less than resolution of the stage
assert stage_axial_resolution < axial_scan_increment, "Axial scan increment is less than resolution of the state!"

# Limit motion stage velocity to the maximum update speed of the light source
stage_axial_velocity = min(axial_scan_increment / illumination_min_update_speed, stage_axial_max_velocity)
print('Stage will be moving at up to %.4f mm/s' % stage_axial_velocity)

# Previous imaging time (single_frame)
axial_accel_time = stage_axial_max_velocity / stage_axial_max_acceleration
axial_accel_distance = stage_axial_max_acceleration * axial_accel_time ** 2
if axial_accel_distance < axial_scan_increment:
    motion_time = axial_scan_increment / stage_axial_max_velocity + stage_axial_max_velocity / stage_axial_max_acceleration + stage_axial_settle_time
else:
    print('WARNING: not reaching maximum velocity')
    motion_time = math.sqrt(axial_scan_increment / stage_axial_max_acceleration) + stage_axial_settle_time
    
frame_time_stop_and_stare = max(camera_readout_time, motion_time) + camera_exposure_time   # mechanical settle time
print('Motion time to readout ratio is %.4f' % (motion_time / camera_readout_time))

# New imaging time (single frame)
# Basiclally we still need to capture enough frames to reconstruct the same amount of data
frame_time_continuous_scan = max(axial_scan_increment / stage_axial_velocity, camera_exposure_time + camera_readout_time) 
if (axial_scan_increment / stage_axial_velocity < stage_axial_velocity, camera_exposure_time + camera_readout_time):
    print('Continuous motion is camera-limited')

print('Frame scan time with existing method (stop and stare) is %.4fs' % frame_time_stop_and_stare)
print('Frame scan time with proposed method (continuous motion) is %.4fs' % frame_time_continuous_scan)
print('Old Acquisition time is %.4f hours, new acquisition time is %.4f hours' % (axial_plane_count * frame_time_stop_and_stare / 3600, axial_plane_count * frame_time_continuous_scan / 3600))
print('Improvement factor (full acquisition) is %.4f' % (frame_time_stop_and_stare / frame_time_continuous_scan))

Stage will be moving at up to 2.0000 mm/s
Motion time to readout ratio is 12.1259
Continuous motion is camera-limited
Frame scan time with existing method (stop and stare) is 0.3061s
Frame scan time with proposed method (continuous motion) is 0.1170s
Old Acquisition time is 0.0513 hours, new acquisition time is 0.0196 hours
Improvement factor (full acquisition) is 2.6166


## Compressed-Sensing Acquisition

In [29]:
# This sets the ratio of how much data we acquire / how much data we construct
compression_factor = 0.1

# Determine how long it takes to zoom through one frame at max velocity
volume_scan_time = sample_size[axial_scan_axis] / stage_axial_velocity

# Determine total acquisition time
frame_time_full_scan = volume_scan_time * compression_factor * axial_plane_count

# Determine the ratio of compressed sensing acquisition
t_acquire_prev = axial_plane_count * frame_time_stop_and_stare
print('Old Acquisition time is %.4f hours, new acquisition time is %.4f hours' % (t_acquire_prev / 3600, frame_time_full_scan / 3600))
print('Improvement factor is %.4f' % (t_acquire_prev / frame_time_full_scan))

Old Acquisition time is 0.0255 hours, new acquisition time is 0.0042 hours
Improvement factor is 6.0814
